In [7]:
import numpy as np
from qiskit import execute, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info.operators import Kraus, SuperOp
from qiskit.providers.aer import QasmSimulator
from qiskit.tools.visualization import plot_histogram
from qiskit.circuit import Parameter

# Qiskit Aer noise module imports
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import QuantumError, ReadoutError
from qiskit.providers.aer.noise.errors import pauli_error
from qiskit.providers.aer.noise.errors import depolarizing_error
from qiskit.providers.aer.noise.errors import thermal_relaxation_error

shots = 8192 
max_credits = 3 

In [38]:
# Create an empty noise model
noise_model = NoiseModel()


e_gate2 = 0.015
e_gate3 = 0.009
e_meas = 0.05
e_cx = 0.06

# Example error probabilities
p_reset = 0.03
p_meas = 0.1
p_gate1 = 0.05

# QuantumError objects
error_reset = pauli_error([('X', p_reset), ('I', 1 - p_reset)])
error_meas = pauli_error([('X',p_meas), ('I', 1 - p_meas)])
error_gate1 = pauli_error([('X',p_gate1), ('I', 1 - p_gate1)])
error_gate2 = error_gate1.tensor(error_gate1)

# Add errors to noise model
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(error_reset, "reset")
noise_model.add_all_qubit_quantum_error(error_meas, "measure")
noise_model.add_all_qubit_quantum_error(error_gate1, ["u1", "u2", "u3"])
noise_model.add_all_qubit_quantum_error(error_gate2, ["cx"])

# Add depolarizing error on qubit 2 forall single qubit u1, u2, u3 gates on qubit 0 
error = depolarizing_error(0.05, 1)
noise_model.add_nonlocal_quantum_error(error, ['u1', 'u2', 'u3'], [0], [2])

print(noise_model)

# System Specification
n_qubits = 3
# Simulator
simulator = QasmSimulator()
# System Specification
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n
circ = QuantumCircuit(q,c)
phi = Parameter('phi')
# Test Circuit
circ.h(q[0])
for i in range(n-1):
    circ.cx(i, i+1)  
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()
for i in reversed(range(n-1)):
    circ.cx(i, i+1)
circ.h(q[0])
circ.barrier()
for i in range(n):
    circ.measure(q[i], c[i]) 
#circ.draw(output='mpl')
print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 


# Ideal execution
job = execute(circ,backend=simulator, parameter_binds=[{phi: phi_val} for phi_val in phi_range],shots=shots)

result_ideal = job.result()

counts_ideal =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('Counts')
print(counts_ideal)
print('Counts of 000')
print(list(map(lambda c: 1/shots*c.get('000', 0), counts_ideal)))

#plot_histogram(counts2)

# Run the noisy simulation
job = execute(circ, backend=simulator,
              basis_gates=noise_model.basis_gates,
              noise_model=noise_model,parameter_binds=[{phi: phi_val} for phi_val in phi_range],shots=shots )

result_thermal = job.result()

counts_thermal =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('Counts')
print(counts_thermal)
print('Counts of 000')
print(list(map(lambda c: 1/shots*c.get('000', 0), counts_thermal)))

# Plot noisy output
#plot_histogram(counts_thermal)



NoiseModel:
  Basis gates: ['cx', 'id', 'u1', 'u2', 'u3']
  Instructions with noise: ['u2', 'reset', 'u1', 'u3', 'cx', 'measure']
  Qubits with noise: [0, 2]
  All-qubits errors: ['reset', 'measure', 'u1', 'u2', 'u3', 'cx']
  Non-local specific qubit errors: [('u1', [0], [2]), ('u2', [0], [2]), ('u3', [0], [2])]
Phi range
[0.0, 0.7853981633974483, 1.5707963267948966, 2.356194490192345, 3.141592653589793, 3.9269908169872414, 4.71238898038469, 5.497787143782138]
Counts
[{'000': 8192}, {'001': 7058, '000': 1134}, {'001': 4097, '000': 4095}, {'001': 1190, '000': 7002}, {'001': 8192}, {'001': 1222, '000': 6970}, {'001': 4024, '000': 4168}, {'001': 6970, '000': 1222}]
Counts of 000
[1.0, 0.138427734375, 0.4998779296875, 0.854736328125, 0.0, 0.850830078125, 0.5087890625, 0.149169921875]
Counts
[{'000': 3486, '101': 280, '010': 1376, '001': 664, '100': 1258, '111': 118, '011': 250, '110': 760}, {'000': 1146, '101': 883, '010': 622, '001': 3002, '100': 646, '111': 451, '011': 987, '110': 455}, 

In [19]:
# T1 and T2 values for qubits 0-3
#parameter inserted for imitate features of ibmq_16_melbourne
T1s = np.random.normal(49.45e3, 19.11e3, 4) # Sampled from normal distribution mean 50 microsec
T2s = np.random.normal(64.33e3, 29.30e3, 4)  # Sampled from normal distribution mean 50 microsec

# Truncate random T2s <= T1s
T2s = np.array([min(T2s[j], 2 * T1s[j]) for j in range(4)])

# Instruction times (in nanoseconds)
time_u1 = 0   # virtual gate
time_u2 = 50  # (single X90 pulse)
time_u3 = 100 # (two X90 pulses)
time_cx = 300
time_reset = 1000  # 1 microsecond
time_measure = 1000 # 1 microsecond

# QuantumError objects
errors_reset = [thermal_relaxation_error(t1, t2, time_reset)
                for t1, t2 in zip(T1s, T2s)]
errors_measure = [thermal_relaxation_error(t1, t2, time_measure)
                  for t1, t2 in zip(T1s, T2s)]
errors_u1  = [thermal_relaxation_error(t1, t2, time_u1)
              for t1, t2 in zip(T1s, T2s)]
errors_u2  = [thermal_relaxation_error(t1, t2, time_u2)
              for t1, t2 in zip(T1s, T2s)]
errors_u3  = [thermal_relaxation_error(t1, t2, time_u3)
              for t1, t2 in zip(T1s, T2s)]
errors_cx = [[thermal_relaxation_error(t1a, t2a, time_cx).expand(
             thermal_relaxation_error(t1b, t2b, time_cx))
              for t1a, t2a in zip(T1s, T2s)]
               for t1b, t2b in zip(T1s, T2s)]

# Add errors to noise model
noise_thermal = NoiseModel()
for j in range(4):
    noise_thermal.add_quantum_error(errors_reset[j], "reset", [j])
    noise_thermal.add_quantum_error(errors_measure[j], "measure", [j])
    noise_thermal.add_quantum_error(errors_u1[j], "u1", [j])
    noise_thermal.add_quantum_error(errors_u2[j], "u2", [j])
    noise_thermal.add_quantum_error(errors_u3[j], "u3", [j])
    for k in range(4):
        noise_thermal.add_quantum_error(errors_cx[j][k], "cx", [j, k])
print(noise_thermal)


# Simulator
simulator = QasmSimulator()

# System Specification
n = 3
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n
circ = QuantumCircuit(q,c)
phi = Parameter('phi')
# Test Circuit
circ.h(q[0])
for i in range(n-1):
    circ.cx(i, i+1)  
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()
for i in reversed(range(n-1)):
    circ.cx(i, i+1)
circ.h(q[0])
circ.barrier()
for i in range(n):
    circ.measure(q[i], c[i]) 
#circ.draw(output='mpl')
print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 


NoiseModel:
  Basis gates: ['cx', 'id', 'u2', 'u3']
  Instructions with noise: ['u2', 'reset', 'u3', 'cx', 'measure']
  Qubits with noise: [0, 1, 2, 3]
  Specific qubit errors: [('reset', [0]), ('reset', [1]), ('reset', [2]), ('reset', [3]), ('measure', [0]), ('measure', [1]), ('measure', [2]), ('measure', [3]), ('u2', [0]), ('u2', [1]), ('u2', [2]), ('u2', [3]), ('u3', [0]), ('u3', [1]), ('u3', [2]), ('u3', [3]), ('cx', [0, 0]), ('cx', [0, 1]), ('cx', [0, 2]), ('cx', [0, 3]), ('cx', [1, 0]), ('cx', [1, 1]), ('cx', [1, 2]), ('cx', [1, 3]), ('cx', [2, 0]), ('cx', [2, 1]), ('cx', [2, 2]), ('cx', [2, 3]), ('cx', [3, 0]), ('cx', [3, 1]), ('cx', [3, 2]), ('cx', [3, 3])]
Phi range
[0.0, 0.7853981633974483, 1.5707963267948966, 2.356194490192345, 3.141592653589793, 3.9269908169872414, 4.71238898038469, 5.497787143782138]


In [15]:
# Ideal execution
job = execute(circ,backend=simulator, parameter_binds=[{phi: phi_val} for phi_val in phi_range],shots=shots)

result_ideal = job.result()

counts_ideal =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('Counts')
print(counts_ideal)
print('Counts of 000')
print(list(map(lambda c: 1/shots*c.get('000', 0), counts_ideal)))

#plot_histogram(counts2)

Counts
[{'000': 8192}, {'001': 6974, '000': 1218}, {'001': 4121, '000': 4071}, {'001': 1207, '000': 6985}, {'001': 8192}, {'001': 1222, '000': 6970}, {'001': 4065, '000': 4127}, {'001': 6995, '000': 1197}]
Counts of 000
[1.0, 0.148681640625, 0.4969482421875, 0.8526611328125, 0.0, 0.850830078125, 0.5037841796875, 0.1461181640625]


In [20]:
# Run the noisy simulation
job = execute(circ, backend=simulator,
              basis_gates=noise_thermal.basis_gates,
              noise_model=noise_thermal,parameter_binds=[{phi: phi_val} for phi_val in phi_range],shots=shots )

result_thermal = job.result()

counts_thermal =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('Counts')
print(counts_thermal)
print('Counts of 000')
print(list(map(lambda c: 1/shots*c.get('000', 0), counts_thermal)))

# Plot noisy output
#plot_histogram(counts_thermal)

Counts
[{'000': 8052, '101': 8, '010': 25, '001': 16, '100': 11, '111': 21, '011': 29, '110': 30}, {'000': 1254, '101': 14, '010': 42, '001': 6793, '100': 18, '111': 18, '011': 32, '110': 21}, {'000': 4053, '101': 8, '010': 41, '001': 3996, '100': 19, '111': 21, '011': 38, '110': 16}, {'000': 6922, '101': 14, '010': 43, '001': 1124, '100': 10, '111': 20, '011': 43, '110': 16}, {'000': 144, '101': 9, '010': 44, '001': 7898, '100': 14, '111': 17, '011': 45, '110': 21}, {'000': 6855, '101': 12, '010': 36, '001': 1207, '100': 12, '111': 16, '011': 41, '110': 13}, {'000': 4036, '101': 16, '010': 29, '001': 4028, '100': 12, '111': 18, '011': 31, '110': 22}, {'000': 1262, '101': 8, '010': 54, '001': 6789, '100': 4, '111': 21, '011': 29, '110': 25}]
Counts of 000
[0.98291015625, 0.153076171875, 0.4947509765625, 0.844970703125, 0.017578125, 0.8367919921875, 0.49267578125, 0.154052734375]
